# Analysis Chain Example

This analysis chain is an example of an **input/output** study. 

1. We draw a sample from the nifty prior distribution
2. generate simulations (using `gen_amp` + `halld_sim/Piecewise` amplitude mimicking prior sample)
3. fit the simulatisons using binned fits (by `AmpTools`) and `iftpwa`

We would like to show a high probability of reconstruction. If this occurs for a wide variety of samples (and we believe our data could be approximately described by the very-flexible nifty prior) then we have successfully demonstrated that inference would be robust. This is in constrast to standard analyses in the field where we come up with ad-hoc models (implicit priors) then perform an I/O study on it. In this case, perhaps only a handful of rigid models are tested which would not necessarily translate to robust inference.

```{note}
This analysis is just for illustration purposes, optimization was not performed for very long.
```

In [ ]:
from pyamptools import atiSetup
import os
os.environ["PYTHONWARNINGS"] = "ignore"

In [2]:
def run_cmd_list(cmds):
    for command in cmds:
        print(f"running: {command}")
        os.system(command)

In [ ]:
atiSetup.setup(globals(), use_fsroot=True, use_genamp=True)

In [ ]:
PYAMPTOOLS_HOME = os.getenv("PYAMPTOOLS_HOME")
subdir = f"{PYAMPTOOLS_HOME}/demos/ift_example/"
print(f"subdir: {subdir}")

In [ ]:
# cleanup
os.system(f'rm -rf {subdir}/NiftyFits')
os.system(f'rm -rf {subdir}/AmpToolsFits')
os.system(f'mkdir -p {subdir}/NiftyFits/prior_sim_DATA')


## Draw prior sample from `iftpwa`

In [ ]:
# Draw from the nifty prior distribution
nifty_prior_draw_cmds = [
    f'sed -i -E "s/force_load_normint: true/force_load_normint: false/" {subdir}/pyamptools.yaml',
    f'mkdir -p {subdir}/NiftyFits/prior_sim',
    f'pa run_ift {subdir}/pyamptools.yaml --prior_simulation',
    f'mv {subdir}/NiftyFits/niftypwa_fit.pkl {subdir}/NiftyFits/prior_sim/niftypwa_fit.pkl',
    f'rm -rf {subdir}/NiftyFits/prior_sim/plots; iftPwaPlot --fitResult {subdir}/NiftyFits/prior_sim/niftypwa_fit.pkl --plotFolder {subdir}/NiftyFits/prior_sim/plots --n_prior_samples 1000'
]

run_cmd_list(nifty_prior_draw_cmds)

## Simulate data from the model and phase space MC

In [ ]:
max_mass = 1.72
min_mass = 1.04
t_slope = 3.0
min_ebeam = 8.2
max_ebeam = 8.8
data_nsamples = 100000
gen_nsamples = 500000

generation_cmds = [
    f'python {subdir}/prior_sim_gen.py', # create the amptools cfg file using halld_sim/Piecewise to model the nifty prior sample
    f'pa gen_amp {subdir}/prior_sim_gen.cfg -o {subdir}/NiftyFits/prior_sim_DATA/data000.root -l {min_mass} -u {max_mass} -n {data_nsamples} -a {min_ebeam} -b {max_ebeam} -t {t_slope}',
    f'mv gen_amp_diagnostic.root {subdir}/NiftyFits/prior_sim_DATA/gen_amp_diagnostic_data.root 2>/dev/null',
    
    f'pa gen_amp {subdir}/prior_sim_gen.cfg -o {subdir}/NiftyFits/prior_sim_DATA/accmc000.root -l {min_mass} -u {max_mass} -n {gen_nsamples} -a {min_ebeam} -b {max_ebeam} -t {t_slope} -f ',
    f'cp {subdir}/NiftyFits/prior_sim_DATA/accmc000.root {subdir}/NiftyFits/prior_sim_DATA/genmc000.root', # uniform acceptance example
    f'mv gen_amp_diagnostic.root {subdir}/NiftyFits/prior_sim_DATA/gen_amp_diagnostic_ps.root 2>/dev/null',
]

run_cmd_list(generation_cmds)

## Run the MLE fits using `AmpTools`

In [ ]:
mle_cmds = [
    f'pa run_cfgGen {subdir}/pyamptools.yaml',
    f'pa run_divideData {subdir}/pyamptools.yaml',
    f'pa run_mle {subdir}/pyamptools.yaml',
    f'pa run_processEvents {subdir}/pyamptools.yaml',
]

run_cmd_list(mle_cmds)

## Run the `iftpwa` fits

In [ ]:
ift_cmds = [
    f'sed -i -E "s/force_load_normint: false/force_load_normint: true/" {subdir}/pyamptools.yaml',
    f'rm -rf {subdir}/NiftyFits/.nifty_expanded.yaml', # trash previous expanded yaml which might not have the right IFT_MODEL.scale since \'auto\' could be overwritten
    f'pa run_ift {subdir}/pyamptools.yaml',
    f"rm -rf {subdir}/NiftyFits/plots",
    f"iftPwaPlot --fitResult {subdir}/NiftyFits/niftypwa_fit.pkl --plotFolder {subdir}/NiftyFits/plots  --massIndepFit {subdir}/NiftyFits/.nifty_expanded.yaml --n_prior_samples 1000",
    f"plotConvergence --inputFolder {subdir}/NiftyFits/fit_parameter_history --plotFolder {subdir}/NiftyFits/convergence_plots",
]

run_cmd_list(ift_cmds)

## Plot the moments and dump all results to a csv file

In [ ]:
aux_cmds = [
    f'pa run_momentPlotter {subdir}/pyamptools.yaml -n 10',
    f'pa run_resultDump {subdir}/pyamptools.yaml -n 10',
]

run_cmd_list(aux_cmds)

## Save copies for the gallery

In [11]:
# # copy some select plots to be used in the gallery and perform cleanup
# copy_cmds = [
#     f'cp {subdir}/NiftyFits/plots/moments/H0\(0,0\)_t0.5.pdf {PYAMPTOOLS_HOME}/assets/images/H0_0_0.pdf',
#     f'cp {subdir}/NiftyFits/plots/moments/H0\(2,0\)_t0.5.pdf {PYAMPTOOLS_HOME}/assets/images/H0_2_0.pdf',
#     f'cp "{subdir}/NiftyFits/plots/parameters/m_{{a_2(1320)}}.pdf" "{PYAMPTOOLS_HOME}/assets/images/m_a2_1320_kde.pdf"',
#     f'cp "{subdir}/NiftyFits/plots/parameters/m_{{a_0(980)}}.pdf" "{PYAMPTOOLS_HOME}/assets/images/m_a0_980_kde.pdf"',
#     f'cp "{subdir}/NiftyFits/convergence_plots/m_{{a_2(1320)}}.pdf" "{PYAMPTOOLS_HOME}/assets/images/m_a2_1320_convergence.pdf"',
#     f'cp "{subdir}/NiftyFits/convergence_plots/m_{{a_0(980)}}.pdf" "{PYAMPTOOLS_HOME}/assets/images/m_a0_980_convergence.pdf"',
# ]

# run_cmd_list(copy_cmds)

# # Copy all pdfs to assets/images and convert to png to be rendered in gallery page
# os.chdir(f'{PYAMPTOOLS_HOME}/assets/images')
# os.system('for f in *.pdf; do convert -density 300 "$f" "${f%.pdf}.png"; done')
# os.system('rm *.pdf')

## Cleanup

In [ ]:
# clean up 
os.system('rm *.pdf')
os.system(f'rm -f {subdir}/../local_beam.conf')
os.system(f'rm -rf {subdir}/AmpToolsFits')
os.system(f'rm -rf {subdir}/NiftyFits')
os.system(f'rm -rf {subdir}/*pkl')
os.system(f'rm -rf {subdir}/*cfg')
os.system(f'rm -rf {subdir}/*csv')